In [30]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import re

import ectools as ec


ModuleNotFoundError: No module named 'ectools'

In [2]:
current_path = os.getcwd()
data_path  = current_path + "\\data\\"

data_files = os.listdir(data_path)

In [3]:
imp = ec.ecImporter()
f = imp.load_folder(data_path)

print(f)

['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_01b_MCL13_cswWE_CV20_initial_test.DTA', 'Format is Gamry']
['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_01c_MCL13_cswWE_CV20_initial_test.DTA', 'Format is Gamry']
['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_01d_MCL13_cswWE_CV20_initial_test.DTA', 'Format is Gamry']
['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_01_MCL13_cswWE_CV20_initial_test.DTA', 'Format is Gamry']
['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_03a_OCP.DTA', 'Format is Gamry']
['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_03_MCL13_cswWE_CV2_750C_CO2.DTA', 'Format is Gamry']
['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_04a_OCP.DTA', 'Format is Gamry']
['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_04_MCL13_cswWE_CV2_750C_CO2.DTA', 'Format is Gamry']
['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_05a_OCP.DTA', 'Format is Gamry']
['-F- c:\\code\\py\\pkg\\ectools\\testing\\data\\240902_

In [4]:
print(f.log)

AttributeError: 'ecList' object has no attribute 'log'

In [41]:

fpath = 'data/'
fname = '240902_18_MCL13_cswWE_CV50_750C_CO2.DTA'

meta_dict = {}
with open(fpath + fname) as f: # Open the file to read the first lines
    while line:=f.readline().strip():
        line = line.split('\t')
        if "TABLE" in line:
            break
        match len(line):
            case 1:
                continue
            case 2: 
                meta_dict[line[0]] = {'value': line[1]}
            case 4 | 5:
                if line[0] == 'NOTES': # handle the multi-line note field
                    meta_dict[line[0]] = {'value': ""}
                    for i in range(0,int(line[2])):
                        meta_dict[line[0]]['value'] += f.readline().strip() # this also accounts for empty lines in notes, so the while loop keeps running
                else:
                    meta_dict[line[0]] = {
                        'label': line[1],
                        'value': line[2],
                        'description': ', '.join(line[3:])
                    }
            case _:
                print('WTF?') # todo raise error
                print(line)

print(meta_dict)


{'TAG': {'value': 'CV'}, 'TITLE': {'label': 'LABEL', 'value': 'Cyclic Voltammetry', 'description': 'Test Identifier'}, 'DATE': {'label': 'LABEL', 'value': '9/3/2024', 'description': 'Date'}, 'TIME': {'label': 'LABEL', 'value': '17:42:40', 'description': 'Time'}, 'NOTES': {'value': '\tcswWE - carbon steel wire\ncspWE - carbon steel plate 2x2 tapered\n\nCarbon steel plate counter electrode\nCarbon steel wire reference electrode\n'}, 'PSTAT': {'label': 'PSTAT', 'value': 'REF3000-48128', 'description': 'Potentiostat'}, 'VINIT': {'label': 'POTEN', 'value': '0.00000E+000', 'description': 'T, Initial E (V)'}, 'VLIMIT1': {'label': 'POTEN', 'value': '1.70000E+000', 'description': 'F, Scan Limit 1 (V)'}, 'VLIMIT2': {'label': 'POTEN', 'value': '-1.00000E+000', 'description': 'F, Scan Limit 2 (V)'}, 'VFINAL': {'label': 'POTEN', 'value': '1.70000E+000', 'description': 'F, Final E (V)'}, 'SCANRATE': {'label': 'QUANT', 'value': '4.99999E+001', 'description': 'Scan Rate (mV/s)'}, 'STEPSIZE': {'label':